In [1]:
import wandb
import pathlib
import urllib
import zstandard
import chess
import torch
import numpy as np
from torch import nn
import math
import time

from torch.nn import init
from torch.utils.data import DataLoader
from torchinfo import summary

In [2]:
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE: " + str(DEVICE))

DEVICE: cuda


In [3]:
wandb.login()

wandb: Currently logged in as: matik (matik001). Use `wandb login --relogin` to force relogin


True

In [4]:
dataset_train = torch.load('dataset_train.save')
dataset_test = torch.load('dataset_test.save')
dataset_valid = torch.load('dataset_valid.save')

loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)
loader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE, shuffle=True)

print("Batches train", len(loader_train))
print("Batches valid", len(loader_valid))
print("Batches test", len(loader_test))

Batches train 12847
Batches valid 500
Batches test 500


In [5]:
DATA_SHAPE = (BATCH_SIZE,) +  tuple(dataset_train[0][0].shape)

In [6]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(*args, **kwargs)


    def init_xavier(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                init.xavier_normal_(m.weight)
    
    def calc_accuracy(self, loader):
        correct = 0
        total = 0
        self.eval()
        with torch.no_grad():
            for batch in loader:
                features, labels = batch
                labels = labels.to(DEVICE) 
                features = features.to(DEVICE)
                outputs = self(features)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0) 
                correct += (predicted == labels).sum().item() 
        self.train()
        return correct / total
    
    def forward(self, X):
        return self.classifier.forward(X)




In [7]:
model0 = Model(nn.Conv2d(21, 8 * 8, kernel_size=4, padding=4),
              nn.ReLU(),
              nn.Conv2d(8 * 8, 4 * 4, kernel_size=2, padding=4),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=4, stride=1),
              nn.Conv2d(4*4, 8 * 8, kernel_size=2, padding=4),
              nn.ReLU(),
              nn.Conv2d(8 * 8, 1, kernel_size=4, padding=4),
              nn.ReLU(),
              nn.Flatten(),
              nn.Linear(841, 256),
              nn.ReLU(),
              nn.Linear(256, 64),
              nn.ReLU(),
              nn.Linear(64, 11)
              )

architecture_summary = summary(model0, DATA_SHAPE)
print(architecture_summary)

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [64, 11]                  --
├─Sequential: 1-1                        --                        --
│    └─Conv2d: 2-1                       [64, 64, 13, 13]          21,568
│    └─ReLU: 2-2                         [64, 64, 13, 13]          --
│    └─Conv2d: 2-3                       [64, 16, 20, 20]          4,112
│    └─ReLU: 2-4                         [64, 16, 20, 20]          --
│    └─MaxPool2d: 2-5                    [64, 16, 17, 17]          --
│    └─Conv2d: 2-6                       [64, 64, 24, 24]          4,160
│    └─ReLU: 2-7                         [64, 64, 24, 24]          --
│    └─Conv2d: 2-8                       [64, 1, 29, 29]           1,025
│    └─ReLU: 2-9                         [64, 1, 29, 29]           --
│    └─Flatten: 2-10                     [64, 841]                 --
│    └─Linear: 2-11                      [64, 256]                 215,5

In [8]:
model1 = Model(nn.Conv2d(21, 300, kernel_size=5, padding=5),
              nn.ReLU(),
              nn.Conv2d(300, 30, kernel_size=2, padding=2),
              nn.ReLU(),
              nn.Flatten(),
              nn.Dropout(0.5), 
              nn.Linear(8670, 20),
              nn.ReLU(),
              nn.Linear(20, 11)
              )

architecture_summary = summary(model1, DATA_SHAPE)
print(architecture_summary)

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [64, 11]                  --
├─Sequential: 1-1                        --                        --
│    └─Conv2d: 2-1                       [64, 300, 14, 14]         157,800
│    └─ReLU: 2-2                         [64, 300, 14, 14]         --
│    └─Conv2d: 2-3                       [64, 30, 17, 17]          36,030
│    └─ReLU: 2-4                         [64, 30, 17, 17]          --
│    └─Flatten: 2-5                      [64, 8670]                --
│    └─Dropout: 2-6                      [64, 8670]                --
│    └─Linear: 2-7                       [64, 20]                  173,420
│    └─ReLU: 2-8                         [64, 20]                  --
│    └─Linear: 2-9                       [64, 11]                  231
Total params: 367,481
Trainable params: 367,481
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 2.66
Input size (MB): 0.34
Forwar

In [9]:
model2 = Model(nn.Conv2d(21, 300, kernel_size=5, padding=5),
              nn.ReLU(),
              nn.Conv2d(300, 30, kernel_size=2, padding=2),
              nn.ReLU(),
              nn.Flatten(),
              nn.BatchNorm1d(8670),
              nn.Linear(8670, 20),
              nn.ReLU(),
              nn.Linear(20, 11)
              )

architecture_summary = summary(model2, DATA_SHAPE)
print(architecture_summary)

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [64, 11]                  --
├─Sequential: 1-1                        --                        --
│    └─Conv2d: 2-1                       [64, 300, 14, 14]         157,800
│    └─ReLU: 2-2                         [64, 300, 14, 14]         --
│    └─Conv2d: 2-3                       [64, 30, 17, 17]          36,030
│    └─ReLU: 2-4                         [64, 30, 17, 17]          --
│    └─Flatten: 2-5                      [64, 8670]                --
│    └─BatchNorm1d: 2-6                  [64, 8670]                17,340
│    └─Linear: 2-7                       [64, 20]                  173,420
│    └─ReLU: 2-8                         [64, 20]                  --
│    └─Linear: 2-9                       [64, 11]                  231
Total params: 384,821
Trainable params: 384,821
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 2.66
Input size (MB): 0.34
Fo

In [10]:
MODELS_LIST = [model0, model1, model2]

In [11]:
def accuracy(out,truth):
    return torch.argmax(out,dim=1) == truth

In [12]:
def train(model, criterion, optimizer, epoch):
    model = model.to(DEVICE)
    model.init_xavier()
    criterion = criterion.to(DEVICE)
    batches_amount = len(loader_train)
    
    for i in range(epoch):
        time_started = time.time() * 1000
        loss_sum=0.0
        accuracy_sum=0.0
        for batch, truth in loader_train:
            batch = batch.to(DEVICE)
            truth = truth.to(DEVICE)
            
            optimizer.zero_grad()
            out = model.forward(batch)
            loss = criterion(out, truth)
            loss.backward()
            optimizer.step()
            
            accuracy_value = accuracy(out,truth).sum()/BATCH_SIZE

            loss_sum+=loss.item()
            accuracy_sum+=accuracy_value.item()

        passed_time = math.ceil(time.time() * 1000 - time_started)
        loss_average = loss_sum/batches_amount
        accuracy_average = accuracy_sum/batches_amount
        validation_accuracy = model.calc_accuracy(loader_valid)

        print(f"Epoch [{i+1}/{epoch}], train_loss: {loss_average}, train_accuracy: {accuracy_average}, validation_accuracy: {validation_accuracy}, time: {passed_time/1000}s")
        wandb.log({
            "epoch": i + 1, 
            "train_loss": loss_average, 
            "train_accuracy" : accuracy_average,
            "validation_accuracy" : validation_accuracy}
        )

In [13]:
### SWEEP CONFIG FOR TESTS
sweep_config = {
    'method': 'bayes',  # random, grid or bayes
    'name': 'sweep-chess-tactics',
    'metric': {
        'goal': 'maximize', 
        'name': 'validation_accuracy'
    },
    'parameters': {
        'epochs': {
            # 'values': [5, 10, 30]
            'values': [20]
        },
        'learning_rate': {
            # 'values': [0.003, 0.001, 0.0003]
            'values': [0.001]
        },
        'optimizer': {
            # 'values': ['sgd', 'adam']
            'values': ['adam']
        },
        'criterion': {
            'values': ['CrossEntropy']
        },
        'model': {
            # 'values': list(range(len(MODELS_LIST)))
            'values': [2]
        }
     },

}

In [ ]:
### MAIN SWEEP CONFIG
sweep_config = {
    'method': 'bayes',  # random, grid or bayes
    'name': 'sweep-chess-tactics',
    'metric': {
        'goal': 'maximize', 
        'name': 'validation_accuracy'
    },
    'parameters': {
        'epochs': {
            # 'values': [5, 10, 30]
            'values': [15, 30]
        },
        'learning_rate': {
            # 'values': [0.003, 0.001, 0.0003]
            'values': [0.0003]
        },
        'optimizer': {
            # 'values': ['sgd', 'adam']
            'values': ['adam']
        },
        'criterion': {
            'values': ['CrossEntropy']
        },
        'model': {
            'values': list(range(len(MODELS_LIST)))
        }
     },

}

In [ ]:
sweep_id = 'cua8zqmh'  # aby kontynuowac trening na poprzednim sweepie

In [14]:
sweep_id = wandb.sweep(sweep_config, project="chess-tactics-swp")

Create sweep with ID: v9ip6m9p
Sweep URL: https://wandb.ai/matik001/chess-tactics-swp/sweeps/v9ip6m9p


In [ ]:


def create_optimizer(model, optimizer):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD
    elif optimizer == "adam":
        optimizer = torch.optim.Adam
    return optimizer

def create_criterion(criterion = "CrossEntropy"):
    if criterion == "CrossEntropy":
        return nn.CrossEntropyLoss()

def main():
    with wandb.init(config=None) as run:
        config = wandb.config
        run.name = f"model{config['model']}_epochs{config['epochs']}_learning_rate{config['learning_rate']}_optimizer{config['optimizer']}"
        
        model = MODELS_LIST[int(config['model'])]
        opt_fn = create_optimizer(model, wandb.config.optimizer)
        crt_fn = create_criterion(wandb.config.criterion)
        train(model, crt_fn, opt_fn(model.classifier.parameters(), lr=wandb.config.learning_rate), wandb.config.epochs)
        model_path = f"{run.name}.save"
        torch.save(model, model_path)


wandb.agent(sweep_id, main)

wandb: Agent Starting Run: cbyi5ef1 with config:
wandb: 	criterion: CrossEntropy
wandb: 	epochs: 20
wandb: 	learning_rate: 0.001
wandb: 	model: 2
wandb: 	optimizer: adam


Epoch [1/20], train_loss: 0.8547805585016431, train_accuracy: 0.7179217424301393, validation_accuracy: 0.76484375, time: 194.965s
Epoch [2/20], train_loss: 0.6235640195220178, train_accuracy: 0.7930316902778859, validation_accuracy: 0.7835625, time: 131.151s


In [ ]:
wandb.finish()

# Testy modelu

In [ ]:
model = torch.load('model1_epochs20_learning_rate0.001_optimizeradam.save')
model.to(DEVICE)

In [ ]:
model.calc_accuracy(loader_train)

In [ ]:
model.calc_accuracy(loader_test)

In [ ]:
model.calc_accuracy(loader_valid)